This page is taking over the functionality of ongoing_workspace.ipynb, so that I can import the data frame as it is currently saved, and run things more efficiently, now that Tasks 1-4 are complete. 

In [1]:
# Preamble - imports 
import numpy as np
import pandas as pd  
import data_cleaning_for_EDA as dc 
import matplotlib.pyplot as plt
import seaborn as sns
import imputing_methods as im 

In [2]:
# Preamble - Multiple Inheritance Class 
class MultipleInheritanceTestClass(im.Plotter, im.DataFrameTransform, dc.DataFrameInfo):
    def export_to_csv(self, filename):
        self.data_frame.to_csv(filename, index=False)
        print(f"DataFrame exported to {filename}")

In [ ]:
# Preamble - data read in
df_df = pd.read_csv("df_end_of_Task_4.csv")
# ensure it's a data frame at this stage so everything inherits correctly in the next step. 
type(df_df)


In [ ]:
df = MultipleInheritanceTestClass(df_df)

## Task 5: Remove outliers  
(Let the record show that as a statistician I do not condone willfully ignoring extereme-yet-possible data! I do so for the purposes of this task.)

Step 1: Visualise with Plotter class to determine if coumns contain outliers. 
I think the best way to indicate that something is an outlier is to plot the actual values against the expected values (were the data Gaussian) in a QQNorm plot. However, since I already plotted loads of this data and identified many non-Gaussian-looking distributions (that's even after "skew adjustment," another statistically questionable practise), I don't think a Normal QQ plot would tell me anything meaningful about outliers. 

I think the best I can offer right now is a box plot. It turs out there's a handy showfliers argument in the sns.boxplot() method-- see Plotter class, where I've added the boxplot_with_outliers() method. 

In [ ]:
# Troubleshooting checks 
print(type(df))

print(type(df.col_names()))

df.stats()



In [ ]:
column_names = df.col_names()

for column in column_names:
        print(f"Plotting boxplot for {column}:")
        df.boxplot_with_outliers(column)


So many of the distributions of these columns struck me as Non-Gaussian to begin with (e.g. exponential or poisson could fit a lot of this data, since we are literally counting "how many accounts have this quality?" and the number of accounts accrues as time passes.), so the concept of using 1.5*IQR as the defining "distance from the quartile" for outliers (which is what sbn.boxplot() does) may not be appropriate. If I see a boxplot has 50 "outliers," they're not really outliers, are they? However, these boxplots give me an initial visual so if I see anything that's truly separate from the pack, I can consider removing it. 

I also won't be removing anything from the id or member_id columns, since these aren't *really* quantitative data. 

So, items to consider removing, depending on how many are there: 


*collection_recovery_fee - has roughly four values higher than 5.5K, but honestly the way the plot tapers suggests exponential/poisson distribution so I wouldn't cut these off willy nilly. 

*recoveries - similar to the above, for values above 17500 

*total_rec_late_fee - because the density is so much more powerful and drops off so suddenly after 150, I'd be more inclined to remove the four or so that are greater than 150 in this column. 

*total_rec_int - there's one that's probably a couple hundred higher than the rest, but the tapering of frequencies shown in this plot suggests another poisson. 

total_accounts - I could consider removing the 5 that are about 85-ish, or just the 2 that are above 90. There's still evidence of tapering which calls the legitimacy of dropping outliers into question. 

open_accounts - clearly discrete dta, with 5 above 45 and 2 above 50, and the thickness of the circles indicates tapering, i.e. a poisson-style distribution. As the values are all fairly close to each other, I wouldn't be inclined to remove any of those. 

*inq_last_6mths - the four above 22 are quite 'gapped' away from the rest, especially the three above 30. 

*deling_2yrs - looks like tapering to me. we'll see how spikey the values above 15 look in a histogram. 

Next I'll examine histograms for the columns with stars, above, to deterimine if in fact the "outliers" the box plot highlighted were isolated and far from the trend, or simply the natural tails of a tapering distribution. 

In [ ]:
spiky_columns = list({"collection_recovery_fee", "recoveries", "total_rec_late_fee", "total_rec_int", "inq_last_6mths","delinq_2yrs"}) 
print(type(spiky_columns))

After much nashing of teeth and growling in an attempt to turn each histogram's y-axis (which is strictly frequency density), a friend suggested I just look at a table to help me judge how far out and unexpected the one-off's are. Let's see if this works... the table() method is added to the Plotter class... 

In [ ]:
for column in spiky_columns: 
    print(f"Table for {column}:")
    df.table(column)

Going to re-examine for a notable gap in some of the tables that were too long to examine from the above. 

In [ ]:
for column in spiky_columns:
        print(f"Plotting boxplot for {column}:")
        df.boxplot_with_outliers(column)

Right, plan of action for outliers has been formed... 

Of those reviewed above, here are my comments on what outliers to remove: 

- collection_recovery_fee - looks like they're all counts of 1 (for a total of just a handful) so just cut off anything bigger than 5000
- recoveries - looks like they're all counts of 1 so cut off anything bigger than 18000
- total_rec_late_fee - cut off anything bigger than 200 
- total_rec_int -  looks like they're all 1's but quite close together for a long time, so cut off the top four, i.e. anything bigger than 21500  
- inq_last_6mths - I can remove the four above 20. 
- deling_2yrs - None of those seem far enough away from a poisson-type distribution to justify dropping off. 

I'll also want to know how many data points I lose as a result of all these cut-offs. I'll be removing roughly 4 from each of the columns above, so if my loss of data points amounts to less than 20, I'll know some of those values were related by virtue of being part of the same data point (which suggests they could have been leverage points, but oh well.). 

In [ ]:
df.print_shape()
df.remove_outlier_rows('collection_recovery_fee', 5000)
df.print_shape() # lost 7

In [ ]:
df.remove_outlier_rows('recoveries', 18000)
df.print_shape() # lost 4

In [ ]:
df.print_shape()
df.remove_outlier_rows('total_rec_late_fee', 200)
df.print_shape() # lost 2

In [ ]:
df.remove_outlier_rows('total_rec_int', 21500)
df.print_shape() # lost 4

In [ ]:
df.remove_outlier_rows('inq_last_6mths', 20)
df.print_shape() # lost 4

It's promising to see from the losses above that there wasn't any overlap, so we moved simple outliers rather than leverage points. 
I'd now like to save this data frame for the next task. 

In [ ]:
df.export_to_csv("df_end_of_Task_5.csv")

## Task 6: Dropping overly correlated columns 

In [3]:
df_2 = pd.read_csv("df_end_of_Task_5.csv")
# ensure it's a data frame at this stage so everything inherits correctly in the next step. 
type(df_2)

pandas.core.frame.DataFrame

In [4]:
df = MultipleInheritanceTestClass(df_2)

First tasked with examining high-correlation pairs of columns. See the corr_matrix() method in DataFrameInfo(), from data_cleaning_for_EDA module. 

In [5]:
# this method will produce a load of pngs to display scatterplots... repo, prepare for a file dump. XD 
df.corr_matrix() #default is 0.65 as a high correlation threshold; I'm happy with that for a start. 

                            Unnamed: 0        id  member_id  loan_amount  \
Unnamed: 0                    1.000000 -0.740928  -0.771418    -0.204667   
id                           -0.740928  1.000000   0.994192     0.134010   
member_id                    -0.771418  0.994192   1.000000     0.139746   
loan_amount                  -0.204667  0.134010   0.139746     1.000000   
funded_amount                -0.214617  0.140651   0.146731     0.964868   
funded_amount_inv            -0.269271  0.168311   0.175817     0.975311   
term_length_in_months        -0.049905  0.026732   0.027910     0.350909   
int_rate                     -0.152098  0.056512   0.060999     0.203116   
instalment                   -0.221409  0.138288   0.144552     0.947076   
years_of_employment          -0.151030  0.079479   0.086432     0.147740   
annual_inc                   -0.046460  0.042588   0.042314     0.368815   
dti                          -0.248654  0.200995   0.203427     0.094046   
delinq_2yrs 

KeyError: "None of [Index([                      ('Unnamed: 0', 'id'),\n                      ('Unnamed: 0', 'member_id'),\n                             ('id', 'Unnamed: 0'),\n                              ('id', 'member_id'),\n                      ('member_id', 'Unnamed: 0'),\n                              ('member_id', 'id'),\n                 ('loan_amount', 'funded_amount'),\n             ('loan_amount', 'funded_amount_inv'),\n                    ('loan_amount', 'instalment'),\n                 ('loan_amount', 'total_payment'),\n             ('loan_amount', 'total_payment_inv'),\n               ('loan_amount', 'total_rec_prncp'),\n                 ('loan_amount', 'total_rec_int'),\n                 ('funded_amount', 'loan_amount'),\n           ('funded_amount', 'funded_amount_inv'),\n                  ('funded_amount', 'instalment'),\n               ('funded_amount', 'total_payment'),\n           ('funded_amount', 'total_payment_inv'),\n             ('funded_amount', 'total_rec_pr